In [1]:
import getpass
import os
import torch
import numpy as np
from PIL import Image
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import torch.nn.functional as F
from collections import OrderedDict
#from skimage import transform
import sys
from sklearn.metrics import roc_curve, auc
import copy
import torchvision.models as models
import re
import xml.etree.ElementTree as ET
import cv2
#from display import read_image, draw_boxes, draw_grid, draw_text
#import display

In [2]:
# with this function you set the value of the environment variable CUDA_VISIBLE_DEVICES
# to set which GPU to use
# it also reserves this amount of memory for your exclusive use. This might be important for 
# not having other people using the resources you need in shared systems
# the homework was tested in a GPU with 4GB of memory, and running this function will require at least
# as much
# if you want to test in a GPU with less memory, you can call this function
# with the argument minimum_memory_mb specifying how much memory from the GPU you want to reserve
def define_gpu_to_use(minimum_memory_mb = 3800):
    gpu_to_use = None
    try: 
        os.environ['CUDA_VISIBLE_DEVICES']
        print('GPU already assigned before: ' + str(os.environ['CUDA_VISIBLE_DEVICES']))
        return
    except:
        pass
    torch.cuda.empty_cache()
    for i in range(16):
        free_memory = !nvidia-smi --query-gpu=memory.free -i $i --format=csv,nounits,noheader
        if free_memory[0] == 'No devices were found':
            break
        free_memory = int(free_memory[0])
        if free_memory>minimum_memory_mb-500:
            gpu_to_use = i
            break
    if gpu_to_use is None:
        print('Could not find any GPU available with the required free memory of ' +str(minimum_memory_mb) + 'MB. Please use a different system for this assignment.')
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_to_use)
        print('Chosen GPU: ' + str(gpu_to_use))
        x = torch.rand((256,1024,minimum_memory_mb-500)).cuda()
        x = torch.rand((1,1)).cuda()
        del x
define_gpu_to_use()

Chosen GPU: 0


In [8]:
def load_raw_images(root,split):
    set_of_transforms = {}
    set_of_transforms['train'] = transforms.Compose(
        [transforms.RandomRotation(30),
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['test'] = transforms.Compose(
        [transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['validate'] = set_of_transforms['test']
    

    whole_dataset = torchvision.datasets.ImageFolder(root=root+'/'+split, transform=set_of_transforms[split])
    
    return whole_dataset    
#path_to_img_folder = '/home/u1135274/test_gpu/DeepLearningProject/color/60_20_20'
path_to_img_folder = '../color/60_20_20'
train_data = load_raw_images(path_to_img_folder,'train')
test_data = load_raw_images(path_to_img_folder,'test')
val_data = load_raw_images(path_to_img_folder,'validate')

print (len(train_data))
print (len(test_data))
print (len(val_data))

16051
6453
4264


In [37]:
train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 84, num_workers = 3)
test_loader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size = 84, num_workers = 3)
val_loader = torch.utils.data.DataLoader(val_data, shuffle = True, batch_size = 84, num_workers = 3)
print (len(train_loader))
print (len(test_loader))
print (len(val_loader))

192
77
51


In [5]:
def read_image(image, size):
    img = cv2.imread(image, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (size, size))
    return img

In [6]:
def draw_boxes(image, boxes_list):
    for boxes in boxes_list:
        cv2.rectangle(image, (int(boxes[0]-boxes[2]/2),int(boxes[1]-boxes[3]/2)),(int(boxes[0]+boxes[2]/2),int(boxes[1]+boxes[3]/2)),(0,0,255),2)
    return image

In [7]:
def draw_grid(img, pixel_step):
    x = pixel_step
    y = pixel_step

    while x < img.shape[1]:
        cv2.line(img, (x, 0), (x, img.shape[0]), color=(255, 255, 255))
        x += pixel_step

    while y < img.shape[0]:
        cv2.line(img, (0, y), (img.shape[1], y), color=(255, 255, 255))
        y += pixel_step

    return img

In [8]:
def draw_text(img, text_list, location):
    for text,loc in zip(text_list, location):
        cv2.putText(img, text, (int(loc[0]), int(loc[1])), cv2.FONT_HERSHEY_COMPLEX,0.5, (255, 0, 0), 1)
    return img

In [15]:
# def img_normalize(image):
#     normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
#     img = normalize(torch.from_numpy(image))
#     return img.numpy()

# def preprocess(img, min_size=600, max_size=1000):
#     C,H,W = img.shape
#     scale1 = min_size/min(H,W)
#     scale2 = max_size/max(H,W)
#     scale = min(scale1, scale2)
#     img = img/255
#     img = transform.resize(img, (C,H*scale, W*scale), mode='reflect', anti_aliasing=False)
#     normalize = img_normalize
#     return normalize(img)

In [14]:
# class CropBiggestCenteredInscribedSquare(object):
#     def __init__(self):
#         pass

#     def __call__(self, tensor):
#         longer_side = min(tensor.size)
#         horizontal_padding = (longer_side - tensor.size[0]) / 2
#         vertical_padding = (longer_side - tensor.size[1]) / 2
#         return tensor.crop(
#             (
#                 -horizontal_padding,
#                 -vertical_padding,
#                 tensor.size[0] + horizontal_padding,
#                 tensor.size[1] + vertical_padding
#             )
#         )

#     def __repr__(self):
#         return self.__class__.__name__ + '()'

In [13]:
# class plant_leaf_disease_dataset(Dataset):
#     def __init__(self, data_dir_bounding_box,data_dir_images,split="train"):
#         self.data_dir_bounding_box = data_dir_bounding_box
#         self.data_dir_images = data_dir_images
#         self.grid_size=16
#         self.label_names = ['AppleScab','CedarAppleRust','CheeryPowderyMildew','FrogEyeSpot','CherrySourPowderyMildew','CornMaizeCercosporaGrayLeafSpot','CornMaizeCommonRust','CornMaizeHealthy','CornMaizeNorthernLeafBlight','GrapeBlackRot','GrapeEscaBlackMeasles','GrapeLeafBlightIsariopsis','OrangeHaungLongbingCitrus','PeachBacterialSpot','PepperBellBacterialSpot','PotatoEarlyBlight','PotatoLateBlight','SquashPowderyMildew','StrawberryLeafScorch','TomatoBacterialSpot','TomatoEarlyBlight','TomatoLateBlight','TomatoLeafMold','TomatoSeptoriaLeafSpot','TomatoSpiderMitesTwoSpotted','TomatoTargetSpot','TomatoMosaicVirus','TomatoYellowLeafCurlVirus']
#         columns=['image_name','label']
#         combined_dataset = []
#         for subdir, dirs, files in os.walk(self.data_dir_bounding_box):
#             for file in files:
#                 #print (file)
#                 #bounding_box, label_name, image_name=self.get_example(file) #ordering xmin, ymin, xmax, ymax
#                 #new_bounding_box = self.convert_to_center(bounding_box)
#                 #print (bounding_box)
#                 record_data = [image_name, label_name] # bounding_box, new_bounding_box, bounding_box[:,0], bounding_box[:,1],bounding_box[:,2], bounding_box[:,3]]
#                 combined_dataset.append(record_data)
#         self.dataset_df = pd.DataFrame(combined_dataset, columns = columns)
# #         self.dataset_df['grid_boxes'] = self.dataset_df['bounding_boxes'].apply(lambda x:self.grid_formation(x))
        
#         image_name_list = pd.unique(self.dataset_df['image_name'])
#         splitted_data = split_data(image_name_list,split)
#         image_name_list = pd.DataFrame(splitted_data,columns=['image_name'])
#         subset_data = pd.merge(image_name_list,self.dataset_df)
#         #print (len(subset_data))
#         self.dataset_df = subset_data
#         self.images = self.dataset_df['image_name'].values
#         self.targets = self.dataset_df['label'].values
# #         self.bounding_boxes = self.dataset_df['bounding_boxes'].values
# #         self.grid_boxes = self.dataset_df['grid_boxes'].values
# #         self.new_bounding_boxes = self.dataset_df['new_bounding_boxes'].values
#         print (len(self.dataset_df))
#         #print (self.dataset_df.head())    
#         #print (self.grid_boxes[0].shape)

    
#     def __getitem__(self, index):
#         set_of_transforms = transforms.Compose(
#         [transforms.RandomRotation(30),
#          transforms.RandomResizedCrop(224),
#          transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(), 
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])])
#         image_val = set_of_transforms(Image.open(self.data_dir_images + '/' + self.targets[index]+'/'+ self.images[index]).convert('RGB'))
#         return self.images[index],image_val, self.label_names.index(self.targets[index])
#     def __len__(self):
#         return len(self.images)        

In [16]:
# path_to_bounding_box = "/home/u1135274/test_gpu/bounding_box/"
# path_to_image_folder = "/home/u1135274/test_gpu/leafDataForBoudingBox/"
# label_names_dataset = ['AppleScab','CedarAppleRust','CheeryPowderyMildew','FrogEyeSpot','CherrySourPowderyMildew','CornMaizeCercosporaGrayLeafSpot','CornMaizeCommonRust','CornMaizeHealthy','CornMaizeNorthernLeafBlight','GrapeBlackRot','GrapeEscaBlackMeasles','GrapeLeafBlightIsariopsis','OrangeHaungLongbingCitrus','PeachBacterialSpot','PepperBellBacterialSpot','PotatoEarlyBlight','PotatoLateBlight','SquashPowderyMildew','StrawberryLeafScorch','TomatoBacterialSpot','TomatoEarlyBlight','TomatoLateBlight','TomatoLeafMold','TomatoSeptoriaLeafSpot','TomatoSpiderMitesTwoSpotted','TomatoTargetSpot','TomatoMosaicVirus','TomatoYellowLeafCurlVirus']
# train_dataset = plant_leaf_disease_dataset(path_to_bounding_box,path_to_image_folder)
# train_loader = torch.utils.data.DataLoader(train_dataset, shuffle = True, batch_size = 28, num_workers = 3)
# validate_dataset = plant_leaf_disease_dataset(path_to_bounding_box,path_to_image_folder,split="validate")
# test_dataset = plant_leaf_disease_dataset(path_to_bounding_box,path_to_image_folder,split="test")
# #torch..utils.data.DataLoader(dataset,)
# validate_loader = torch.utils.data.DataLoader(validate_dataset, shuffle = True, batch_size = 28, num_workers = 3)
# test_loader = torch.utils.data.DataLoader(test_dataset, shuffle = True, batch_size = 28, num_workers = 3)
# print (len(test_dataset))

In [9]:
# i = train_dataset.__getitem__(0)
# img=i[0]
# print (i[2])
# img = read_image(path_to_image_folder+"/"+label_names_dataset[i[2]]+"/"+img, 256)
# img = draw_boxes(img, i[3])
# print (i[4])
# img = draw_text(img, label_names_dataset[i[2]], i[3])
# plt.imshow(img)
# plt.show()


In [32]:
classification_model = models.resnet152(pretrained=True)
for param in classification_model.parameters():
    param.requires_grad = False
print (classification_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [33]:
fc_layer = torch.nn.Sequential(OrderedDict([
    ('fc_layer1',torch.nn.Linear(2048, 512)),
    ('relu',torch.nn.ReLU()),
    ('fc_layer2',torch.nn.Linear(512,38)),
    ('softmax',torch.nn.LogSoftmax(dim=1))]))
classification_model.fc = fc_layer

In [50]:
def train_classification_model(model, criterion, optimizer, lr_decay, num_epochs=20,mode='train'):
    for epoch in range(num_epochs):
        if mode=='train':
            lr_decay.step()
            model.train()
        elif mode=='val':
            model.eval()
        loss_sum = []; acc = 0
        for i in train_loader: #change later
            input_img =i[0].cuda()#change later
            target = i[1].cuda() #change later
            optimizer.zero_grad()
            with torch.set_grad_enabled(mode=='train'):
                predicted_output = model(input_img)
                loss = criterion(predicted_output, target)
                _,pred_out = torch.max(predicted_output,1)
                
                if mode=='train':
                    loss.backward()
                    optimizer.step()
            loss_sum.append(loss.item())
            acc+=torch.sum(pred_out==target.data)
        epoch_loss = sum(loss_sum)/len(loss_sum)
        epoch_accuracy = acc.item()/len(train_data)
        print ("epoch:"+str(epoch))
        print ("Loss:{:.6f}".format(epoch_loss))
        print ("Accuracy:{:.6f}".format(epoch_accuracy))

#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.fc.parameters(), lr=0.001)
lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
train_classification_model(classification_model, criterion, optimizer,lr_schedule,5,'train')

epoch:0
Loss:0.293291
Accuracy:0.904928
epoch:1
Loss:0.287164
Accuracy:0.910847
epoch:2
Loss:0.278327
Accuracy:0.912653
epoch:3
Loss:0.257370
Accuracy:0.915083
epoch:4
Loss:0.268209
Accuracy:0.913899


In [52]:
train_classification_model(classification_model, criterion, optimizer,lr_schedule,1,'val')

epoch:0
Loss:0.237396
Accuracy:0.923494


In [51]:
#test
def test_model(model):
    model.eval()
    accuracy_list=[]
    for i in val_loader:
        input_img=i[0].cuda()
        target=i[1].cuda()
        pred_output = model(input_img)
        eq = (target.data == torch.exp(pred_output).max(1)[1])
        accuracy_list.append(eq.type_as(torch.FloatTensor()).mean())
    print ("Test accuracy:{:.6f}".format(sum(accuracy_list)/len(accuracy_list)))
test_model(classification_model)

Test accuracy:0.930687
